In [ ]:
# !pip install tensorflow==2.11.0
# !pip install tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import os, glob
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models

# 이미지 불러오기

In [ ]:
def load_images_from_folder(folder, target_size=(256, 256)):
    paths = sorted(glob.glob(os.path.join(folder, '*.png')))
    images = []
    for path in paths:
        img = Image.open(path).convert('L').resize(target_size)  # 이미지를 그레이스케일로 변환하고 크기 조정
        img_array = np.array(img) / 255.0 # 0-1 범위로 정규화
        images.append(img_array[..., np.newaxis])  # (128,128,1) # 채널 차원 추가 (128,128,1)
    return np.array(images)

image_folder = "drive/MyDrive/Colab Notebooks/Project2/Model/img256"
images = load_images_from_folder(image_folder)
print("Loaded image shape:", images.shape)  # 이미지 shape 출력 : (n, 128, 128, 1)

# Autoencoder 정의 및 학습

In [ ]:
def build_autoencoder():
    # 인코더
    encoder_input = layers.Input(shape=(256, 256, 1))
    x = layers.Conv2D(32, 3, strides=2, activation='relu', padding='same')(encoder_input)
    x = layers.Conv2D(64, 3, strides=2, activation='relu', padding='same')(x)
    x = layers.Flatten()(x)
    encoded = layers.Dense(100)(x)

    # 디코더
    x = layers.Dense(64 * 64 * 64, activation='relu')(encoded)
    x = layers.Reshape((64, 64, 64))(x)
    x = layers.Conv2DTranspose(64, 3, strides=2, activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(32, 3, strides=2, activation='relu', padding='same')(x)
    decoder_output = layers.Conv2DTranspose(1, 3, activation='sigmoid', padding='same')(x)

    autoencoder = models.Model(encoder_input, decoder_output)
    encoder = models.Model(encoder_input, encoded)
    return autoencoder, encoder

autoencoder, encoder = build_autoencoder() # 모델 생
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

In [ ]:
# 학습
autoencoder.fit(images, images, epochs=20, batch_size=32)

# 이미지 → 벡터 인코딩 및 시계열 구성

In [ ]:
latents = encoder.predict(images) # 인코더를 통해 잠재 변수(latent) 추출

# 잠재 변수들을 시계열 데이터로 변환
def create_sequences(data, seq_len=12):
    sequences = []
    for i in range(len(data) - seq_len):
        sequences.append(data[i:i+seq_len])
    return np.array(sequences)

seq_data = create_sequences(latents, seq_len=12)
print("TimeGAN input shape:", seq_data.shape)

# LSTM Generator

In [ ]:
# LSTM Generator 모델
def build_lstm_generator(seq_len=12, latent_dim=100):
    model = models.Sequential([
        layers.Input(shape=(seq_len, latent_dim)),
        layers.LSTM(128, return_sequences=True),
        layers.LSTM(100, return_sequences=False),
        layers.Dense(latent_dim)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

lstm_gen = build_lstm_generator()
X = seq_data[:, :-1, :] # 입력 시퀀스
y = seq_data[:, -1, :] # 타겟 값
lstm_gen.fit(X, y, epochs=100, batch_size=1)

# 생성된 벡터 → 이미지 디코딩 및 시각화

In [ ]:
pred_latent = lstm_gen.predict(X) # 예측된 잠재 변수

# 디코더 모 정의
decoder_input = layers.Input(shape=(100,))
x = layers.Dense(64 * 64 * 64, activation='relu')(decoder_input)
x = layers.Reshape((64, 64, 64))(x)
x = layers.Conv2DTranspose(64, 3, strides=2, activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(32, 3, strides=2, activation='relu', padding='same')(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation='sigmoid', padding='same')(x)
decoder = models.Model(decoder_input, decoder_output)

# 오토인코더의 가중치를 디코더에 전달
for i in range(len(decoder.layers)):
    try:
        decoder.layers[i].set_weights(autoencoder.layers[-(len(decoder.layers)-i)].get_weights())
    except:
        pass


reconstructed_imgs = decoder.predict(pred_latent)
print(reconstructed_imgs.shape)

# 이미지 복원 비교

In [ ]:
# MSE (Mean Squared Error) 계산
from sklearn.metrics import mean_squared_error

index = 0
real_image = images[index+12].squeeze()  # 실제 이미지
predicted_image = reconstructed_imgs[index].squeeze()  # 예측 이미지

# MSE 계산
mse = mean_squared_error(real_image.flatten(), predicted_image.flatten())

max_pixel_value = 1.0
difference_percentage = mse / (max_pixel_value ** 2) * 100

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(images[index+12].squeeze(), cmap='gray')  # 실제
plt.title('Real Next Image')

plt.subplot(1, 2, 2)
plt.imshow(reconstructed_imgs[index].squeeze(), cmap='gray')  # 예측
plt.title(f'Predicted Image \n Difference Percentage:{difference_percentage:.2f}%')
plt.show()
